* [INDEX ]

* [Data EXPLORations](#1)
* [Data Cleaning Steps](#2)
* [Data Preprocess_Ops](#3)
* [Pipeline. ](#4)

# Data EXPLORations<a class="anchor" id="1"></a>

In [1]:
import findspark
findspark.init("/Users/resitkadir/spark/spark-3.0.0/")
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[4]") \
        .appName("DataCleaning") \
        .config("spark.executor.memory","8g") \
        .config("spark.driver.memory","2g") \
        .getOrCreate()


# VERİ SETİNİ OKUMA ÖNCESİ KONTROLLER
1. Veri setine ulaşma:
    adult.data ve adult.test adında iki veri dosyası var.
2. Mümkünse Notepad++ ile içine bakalım. Baktık. virgülle ayrılmış ve başlıkları olan bir dosya

In [2]:
adult_train_df = spark.read \
                .option("header","True") \
                .option("inferSchema","True") \
                .option("sep",",") \
                .csv("datasets/adult/adult.data") \

adult_test_df = spark.read \
                .option("header","True") \
                .option("inferSchema","True") \
                .option("sep",",") \
                .csv("datasets/adult/adult.test") \

adult_train_df.show(3)

+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-----+------------+------------+--------------+--------------+------+
|age|        workclass|  fnlwgt| education|education_num|     marital_status|        occupation|  relationship|  race|  sex|capital_gain|capital_loss|hours_per_week|native_country|output|
+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-----+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516.0| Bachelors|         13.0|      Never-married|      Adm-clerical| Not-in-family| White| Male|      2174.0|         0.0|          40.0| United-States| <=50K|
| 50| Self-emp-not-inc| 83311.0| Bachelors|         13.0| Married-civ-spouse|   Exec-managerial|       Husband| White| Male|         0.0|         0.0|          13.0| United-States| <=50K|
| 38|          Private|215646.0|   HS-grad|          9.0|   

    DF çok geniş olduğu için düzgün bir görünüm alamadık. Driver'a aldığımız birkaç satırdan oluşan dataframe'i pandas dataframe'e çevirerek daha güzel bir görünüm sağlayabiliriz.

In [3]:
adult_train_df.limit(3).toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K


In [4]:
adult_test_df.limit(3).toPandas().head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,output
0,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
1,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
2,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.


In [5]:
adult_whole_df = adult_train_df.union(adult_test_df)
adult_whole_df.limit(3).toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K


In [6]:
print("train satır sayısı: ", adult_train_df.count())
print("test satır sayısı:  ", adult_test_df.count())
print("Total satır sayısı: ", adult_whole_df.count())

train satır sayısı:  32561
test satır sayısı:   16281
Total satır sayısı:  48842


In [7]:
#Satır sayıları tutarlı olduğu için başlık yanlışlıkla whale_df içine girmemiş. Aferin union'a.

### VERİ SETİNİ İNCELEME ŞEMA İLE KARŞILAŞTIRMA

In [8]:
adult_whole_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- output: string (nullable = true)



### NÜMERİK DEĞİŞKENLERİN İSTATİSTİKLERİ

In [9]:
adult_whole_df.describe(['age','fnlwgt','education_num','capital_gain','capital_loss','hours_per_week']) \
.toPandas().head()

,summary,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,count,48842,48842,48842,48842,48842,48842
1,mean,38.64358543876172,189664.13459727284,10.078088530363212,1079.0676262233324,87.50231358257237,40.422382375824085
2,stddev,13.710509934443563,105604.02542315733,2.570972755592263,7452.019057655401,403.0045521243599,12.3914440242523
3,min,17,12285.0,1.0,0.0,0.0,1.0
4,max,90,1490400.0,16.0,99999.0,4356.0,99.0


## KATEGORİK DEĞİŞKENLERİN İNCELENMESİ

    Kategorik değişkenlerin incelenmesinde groupBy() kullanmak daha çok bilgi verir. Hedef değişken dahil toplam 9 kategorik değişken var.

In [10]:
## 1. workclass

from pyspark.sql import functions as F
adult_whole_df.groupBy(F.col("workclass")) \
                        .agg({"*":"count"}) \
                        .toPandas().head(10)

,workclass,count(1)
0,State-gov,1981
1,Federal-gov,1432
2,Self-emp-not-inc,3862
3,Local-gov,3136
4,Private,33906
5,?,2799
6,Self-emp-inc,1695
7,Without-pay,21
8,Never-worked,10


In [11]:
adult_whole_df.createOrReplaceTempView("workclass_tbl")
spark.sql(" SELECT workclass, COUNT(*) FROM workclass_tbl GROUP BY workclass LIMIT 10") \
.toPandas().head(2)

,workclass,count(1)
0,State-gov,1981
1,Federal-gov,1432


    Yorum: 2.799 adet ? var. Bu nedir. Muhtelemen kayıp bilgi.Daha sonra never-worked ve without-pay sınıfları çok az tekrarlanmış. Bunların da veri setinden çıkarılması düşünebilir

In [12]:
# 2. education
adult_whole_df.groupBy(F.col("education")) \
            .agg({"*":"count"}) \
            .toPandas().head(4)

,education,count(1)
0,Prof-school,834
1,10th,1389
2,7th-8th,955
3,5th-6th,509


Yorum: Genel bir sıkıntı görünmüyor ancak çok fazla kategori var belki bazıları birleştirilebilir.

      1st-4th, 5th-6th, 7th-8th: elementary-school

      9th, 10th, 11th, 12th: high-school

      Masters, Doctorate: high-education

      Bachelors, Some-college: undergraduate

In [13]:
#3. marital_status¶
adult_whole_df.groupBy(F.col("marital_status")) \
            .agg({"*":"count"}) \
            .toPandas().head(4)

,marital_status,count(1)
0,Widowed,1518
1,Married-spouse-absent,628
2,Married-AF-spouse,37
3,Married-civ-spouse,22379


In [14]:
# 4. occupation

adult_whole_df.groupBy(F.col("occupation")) \
        .agg({"*":"count"}) \
        .toPandas().head(2)

,occupation,count(1)
0,Farming-fishing,1490
1,Handlers-cleaners,2072


    Yorum: 2809 tane ? var. Bunlar muhtemelen bilinmeyenler. Ayrıca Armed-Forces 15 kişi. Bu sınıfa ait kayıtlar çıkarılabilir.

In [15]:
# 5. relationship
adult_whole_df.groupBy(F.col("relationship")) \
        .agg({"*":"count"}) \
        .toPandas().head(2)

,relationship,count(1)
0,Husband,19716
1,Own-child,7581


In [16]:
#Race
adult_whole_df.groupBy(F.col("race")) \
        .agg({"*":"count"}) \
        .toPandas().head(2)

,race,count(1)
0,Asian-Pac-Islander,1519
1,Black,4685


In [17]:
#7-SEX
adult_whole_df.groupBy(F.col("sex")) \
    .agg({"*":"count"}) \
    .toPandas().head(2)

,sex,count(1)
0,Male,32650
1,Female,16192


In [18]:
#8 native Country
adult_whole_df.groupBy(F.col("native_country")) \
            .agg({"*":"count"}) \
            .toPandas().head(4)

,native_country,count(1)
0,Dominican-Republic,103
1,Ireland,37
2,Cuba,138
3,Guatemala,88


In [19]:
#9 OutPut
adult_whole_df.groupBy(F.col("output")) \
        .agg({"*":"count"}) \
        .show()

+-------+--------+
| output|count(1)|
+-------+--------+
|   >50K|    7841|
|  >50K.|    3846|
| <=50K.|   12435|
|  <=50K|   24720|
+-------+--------+



In [20]:
spark.stop()

/////////////////////// VERİ TEMİZLİĞİ İÇİN TAVSİYELER ////////////////////////////////////////////////

  1. Tüm sütunları boşluk kontrolü yap.
  2. ? içeren workclass, occupation  var bunların ? içerdiği satırlar tekrar incelenmeli.
      ? işaretleri sistematik bir şekilde mi oluşmuş yoksa bu oluşum tesadüfi mi?
      ? kayıtlarının oluşması altında yatan bir mekanizma var mı?
      Bu sistematik hata yakalanırsa veri doldurma (imputation) yoksa satır silme yapılsın.
  3. workclass niteliğinde never-worked ve without-pay sınıfları ve
      occupation niteliğinde  Armed-Forces  sınıfı
    çok az tekrarlanmış. Veri setinden çıkarılabilir.
  4. education niteliğindeki:
          1st-4th, 5th-6th, 7th-8th: elementary-school
          9th, 10th, 11th, 12th: high-school
          Masters, Doctorate: high-education
          Bachelors, Some-college: undergraduate
     sınıfları yukarıdaki gibi birleştirilebilir.
  5. native country'de ? var ve Hollanda sadece 1 kez tekrarlanmış. 
  6. output (hedef değişkendeki) >50K. ve <=50K. değerlerindeki "." kaldırılmalı

# DATA CLEANING<a class="anchor" id="2"></a>

/////////////////////// VERİ TEMİZLİĞİ İÇİN TAVSİYELER  ////////////////////////////////////////////////
    /*
      1. Tüm sütunları boşluk kontrolü yap.
      2. ? içeren workclass, occupation  var bunların ? içerdiği satırlar tekrar incelenmeli.
          ? işaretleri sistematik bir şekilde mi oluşmuş yoksa bu oluşum tesadüfi mi?
          ? kayıtlarının oluşması altında yatan bir mekanizma var mı?
          Bu sistematik hata yakalanırsa veri doldurma (imputation) yoksa satır silme yapılsın.
      3. workclass niteliğinde never-worked ve without-pay sınıfları ve
          occupation niteliğinde  Armed-Forces  sınıfı
        çok az tekrarlanmış. Veri setinden çıkarılabilir.
      4. education niteliğindeki:
              1st-4th, 5th-6th, 7th-8th: elementary-school
              9th, 10th, 11th, 12th: high-school
              Masters, Doctorate: high-education
              Bachelors, Some-college: undergraduate
         sınıfları yukarıdaki gibi birleştirilebilir.
      5. native_country'de ? var ve Hollanda 1 kez tekrarlanmış.
      6. output (hedef değişkendeki) >50K. ve <=50K. değerlerindeki "." kaldırılmalı
     */

In [21]:
import findspark
findspark.init("/Users/resitkadir/spark/spark-3.0.0/")
from pyspark.sql import SparkSession


spark = SparkSession.builder \
        .master("local[4]") \
        .appName("DataCleaning") \
        .config("spark.executor.memory","8g") \
        .config("spark.driver.memory","2g") \
        .getOrCreate()

#Read The data

adult_train_df = spark.read \
                .option("header","True") \
                .option("inferSchema","True") \
                .option("sep",",") \
                .csv("datasets/adult/adult.data") \

adult_test_df = spark.read \
                .option("header","True") \
                .option("inferSchema","True") \
                .option("sep",",") \
                .csv("datasets/adult/adult.test") \

#Veri birlestirme
adult_whole_df = adult_train_df.union(adult_test_df)
adult_whole_df.limit(3).toPandas().head()




,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K


In [22]:
#1. Tüm sütunları boşluk kontrolü yap.
from pyspark.sql.functions import *

kategorik_nitelikler = ["workclass", "education", "marital_status", "occupation", "relationship", "race",
          "sex", "native_country", "output"]
numerik_nitelikler = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]


In [23]:
#Boşluk temizleme (trim) sadece string değişkenlere uygulanabilir.
adult_whole_df1 = adult_whole_df \
.withColumn("workclass", trim(col("workclass"))) \
.withColumn("education", trim(col("education"))) \
.withColumn("marital_status", trim(col("marital_status"))) \
.withColumn("occupation", trim(col("occupation"))) \
.withColumn("relationship", trim(col("relationship"))) \
.withColumn("race", trim(col("race"))) \
.withColumn("sex", trim(col("sex"))) \
.withColumn("native_country", trim(col("native_country"))) \
.withColumn("output", trim(col("output")))
adult_whole_df.count()

48842

In [24]:
adult_whole_df1.count()

48842

#  2. OUTPUT İÇİNDEKİ "." TEMİZLİĞİ 
 Her ne kadar taslak planda output temizliğini son madde olarak yazmış olsak da. İşin kolay olması ve diğer temizlik
işlemlerini etkiliyor olması nedeniyle. Bu temizliği en başta yapalım.

    // . içerenleri filtreleyelim ancak "~" ile tersini yapalım.

In [25]:
adult_whole_df2 = adult_whole_df1 \
        .withColumn("output", regexp_replace(col("output"), "<=50K.","<=50K")) \
        .withColumn("output", regexp_replace(col("output"), ">50K.",">50K"))

# Temizlik sonucunu görelim.
adult_whole_df2.groupBy(col("output")).agg({"*":"count"}) \
                .toPandas().head()


,output,count(1)
0,<=50K,37155
1,>50K,11687


### 3. NULL KONTROLÜ

        // Her bir sütun için for dongüsü ile o sütunda filter() metodu ile null kontrolü yapalım ve bunları sayalım. Eğer o sütunda en az bir adet null varsa yani >0 ise o sütunda null olduğunu ekrana yazdıralım.

In [26]:
adult_whole_df2.columns

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'output']

In [27]:
sayac_for_null = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).isNull()).count() > 0):
        print(sayac_for_null, ". ", sutun, " içinde null değer var.")
    else:
        print(sayac_for_null, ". ", sutun)
    sayac_for_null+=1
        # Herhangi bir null yok.

1 .  age
2 .  workclass
3 .  fnlwgt
4 .  education
5 .  education_num
6 .  marital_status
7 .  occupation
8 .  relationship
9 .  race
10 .  sex
11 .  capital_gain
12 .  capital_loss
13 .  hours_per_week
14 .  native_country
15 .  output


### 4.  "?" KONTROLLERİ

In [28]:
sayac_for_question = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).contains("?")).count() > 0):
        print(sayac_for_question, ". ", sutun, " içinde ? var.")
    else:
        print(sayac_for_question, ". ", sutun)
    sayac_for_question+=1
    
    
        #workclass, occupation, native_country içinde ? var.
    
    #Bunların hedef değişken output ile ilişkisini inceleyelim.

1 .  age
2 .  workclass  içinde ? var.
3 .  fnlwgt
4 .  education
5 .  education_num
6 .  marital_status
7 .  occupation  içinde ? var.
8 .  relationship
9 .  race
10 .  sex
11 .  capital_gain
12 .  capital_loss
13 .  hours_per_week
14 .  native_country  içinde ? var.
15 .  output


In [29]:
adult_whole_df2.select("workclass", "occupation", "native_country", "output") \
.filter(col("workclass").contains("?") | col("occupation").contains("?") | col("native_country").contains("?")) \
.groupBy("workclass", "occupation", "native_country", "output").count() \
.orderBy(col("count").desc()) \
.toPandas().head(5)

,workclass,occupation,native_country,output,count
0,?,?,United-States,<=50K,2284
1,?,?,United-States,>50K,246
2,Private,Other-service,?,<=50K,100
3,Private,Sales,?,<=50K,55
4,Private,Prof-specialty,?,<=50K,51


"""
    // Soru işaretlerinin dağılımı ve hedef değişken ile ilgisi tesadüfi görünüyor. Bu durumda ? işareti içeren satırları veri setinden çıkaralım.

"""

In [30]:
adult_whole_df3 = adult_whole_df2 \
        .filter(~(col("workclass").contains("?") | col("occupation").contains("?") | col("native_country").contains("?")))
print(adult_whole_df2.count())
print(adult_whole_df3.count())
#// adult_whole_df2'den adult_whole_df3'e geçerken ? işareti olan satırlar silindi.

48842
45222


### 5. ZAYIF SINIFLARIN KALDIRILMASI

     // workclass niteliğinde never-worked ve without-pay sınıfları ve
    occupation niteliğinde  Armed-Forces  sınıfı
      çok az tekrarlanmış. Veri setinden çıkarılabilir.

In [31]:
adult_whole_df4 = adult_whole_df3 \
                .filter(~(col("workclass").contains("never-worked") | col("workclass").contains("without-pay") |
                col("occupation").contains("Armed-Forces")))

print(adult_whole_df3.count())
print(adult_whole_df4.count())

#    // adult_whole_df3'ten adult_whole_df4'e geçerken zayıf sınıfların bulunduğu satırlar silindi.

45222
45208


### 6. EĞİTİM DURUMUYLA İLGİLİ KATEGORİLERİ BİRLEŞTİRME

    // education niteliğindeki:
      1st-4th, 5th-6th, 7th-8th: elementary-school
    9th, 10th, 11th, 12th: high-school
    Masters, Doctorate: high-education
    Bachelors, Some-college: undergraduate
    sınıfları yukarıdaki gibi birleştirilebilir.
    
  ## Uyarı: Aşağıdaki yöntem trim edilmemiş niteliklerde kullanılmamalıdır.

In [32]:
adult_whole_df5 = adult_whole_df4.withColumn("education_merged", \
      when(col("education").isin("1st-4th","education","5th-6th","education","7th-8th"), "Elementary-School")
        .when(col("education").isin("9th","10th","11th","12th"), "High-School")
        .when(col("education").isin("Masters","Doctorate"), "Post-Graduate")
        .when(col("education").isin("Bachelors","Some-college"), "Under-Graduate")
        .otherwise(col("education")))
adult_whole_df5.select("education","education_merged").toPandas().head(20)


,education,education_merged
0,Bachelors,Under-Graduate
1,Bachelors,Under-Graduate
2,HS-grad,HS-grad
3,11th,High-School
4,Bachelors,Under-Graduate
5,Masters,Post-Graduate
6,9th,High-School
7,HS-grad,HS-grad
8,Masters,Post-Graduate
9,Bachelors,Under-Graduate


In [33]:
adult_whole_df5.groupBy("education_merged").agg({"*":"count"}).toPandas().head(20)
#    // Evet yeni bir sütun ekledik. education_merged. Daha sade.

,education_merged,count(1)
0,Assoc-acdm,1507
1,Assoc-voc,1959
2,Elementary-School,1494
3,High-School,4094
4,HS-grad,14778
5,Preschool,72
6,Post-Graduate,3056
7,Under-Graduate,17464
8,Prof-school,784


### 7. native_country niteliğinde temizlik

    // native_country'de ? var ve Hollanda 1 kez tekrarlanmış.
    // ? temizleme işini 4. aşamada hallettik.
    // Burada sadece Hollandayı (Holand-Netherlands) çıkaralım.

In [34]:
adult_whole_df6 = adult_whole_df5.filter(~(col("native_country") == "Holand-Netherlands"))
print(adult_whole_df5.count())
print(adult_whole_df6.count())
#Satır bir azaldığına göre çıkarma işlemi başarılı.

45208
45207


### 8. OUTPUT . KALDIRMA
     // Bu temizliği 2. aşamada yapmıştık.

### 9. TEMİZ VERİYİ DİSKE YAZALIM
    // Sütun sırasını değiştirelim. education_merge'i education yanına alalım

In [35]:
nitelik_siralama = ["workclass", "education", "education_merged", "marital_status", "occupation", "relationship", "race",
     "sex", "native_country", "age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week","output"]
adult_whole_df7 = adult_whole_df6.select(nitelik_siralama)

adult_whole_df7.toPandas().head()

,workclass,education,education_merged,marital_status,occupation,relationship,race,sex,native_country,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,output
0,State-gov,Bachelors,Under-Graduate,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,39,77516.0,13.0,2174.0,0.0,40.0,<=50K
1,Self-emp-not-inc,Bachelors,Under-Graduate,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,50,83311.0,13.0,0.0,0.0,13.0,<=50K
2,Private,HS-grad,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,38,215646.0,9.0,0.0,0.0,40.0,<=50K
3,Private,11th,High-School,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,53,234721.0,7.0,0.0,0.0,40.0,<=50K
4,Private,Bachelors,Under-Graduate,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,28,338409.0,13.0,0.0,0.0,40.0,<=50K


In [36]:
 adult_whole_df7 \
      .coalesce(1) \
      .write \
      .mode("overwrite") \
      .option("sep",",") \
      .option("header","true") \
      .csv("datasets\adult_preprocessed_pyspark")

In [37]:
spark.stop()

# Data Preprocessing <a class="anchor" id="3"></a>

In [61]:
import findspark
findspark.init("/Users/resitkadir/spark/spark-3.0.0/")
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[4]") \
        .appName("PreprocessOps") \
        .config("spark.executor.memory","4g") \
        .config("spark.driver.memory","2g") \
        .getOrCreate()

df = spark.read \
        .option("header","True") \
        .option("inferSchema","True") \
        .option("sep",",") \
        .csv("datasets/simple_data.csv")

df.toPandas().head()

,sirano,isim,yas,meslek,sehir,aylik_gelir
0,1,Cemal,35,Isci,Ankara,3500
1,2,Ceyda,42,Memur,Kayseri,4200
2,3,Timur,30,Müzisyen,Istanbul,9000
3,4,Burcu,29,Pazarlamaci,Ankara,4200
4,5,Yasemin,23,Pazarlamaci,Bursa,4800


###  VERİ SETİNE ETİKET EKLEME 

    // Sınıflandırma hedef değişken (etiket-label) yaratmak adına

    // Geliri 7000 üstü olanların ekonomik_durumu iyi diyelim.

In [62]:
import pyspark.sql.functions as F

df1 = df.withColumn("ekonomik_durum",
        F.when(F.col("aylik_gelir") >= 7000, "iyi").otherwise("kötü"))

df1.toPandas().head()

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum
0,1,Cemal,35,Isci,Ankara,3500,kötü
1,2,Ceyda,42,Memur,Kayseri,4200,kötü
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü


In [63]:
#   1. StringIndexer AŞAMASI 
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
## meslek sütunu için StringIndexer
meslek_indexer = StringIndexer() \
                .setInputCol("meslek") \
                .setOutputCol("meslek_index") \
                .setHandleInvalid("skip") 

meslek_indexer_model = meslek_indexer.fit(df1)
meslek_idexer_df = meslek_indexer_model.transform(df1)
meslek_idexer_df.toPandas().head(5)



,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0


In [65]:
## Mesleklerin frekansları:
df1.groupBy(F.col("meslek")) \
        .agg(F.count("*").alias("sayi")) \
        .sort(F.desc("sayi")) \
        .toPandas().head(10)

,meslek,sayi
0,Memur,3
1,Müzisyen,3
2,Pazarlamaci,3
3,Doktor,2
4,Isci,1
5,Tuhafiyeci,1
6,Berber,1
7,Tornacı,1


In [42]:
## sehir sütunu için StringIndexer

sehir_indexer = StringIndexer() \
.setInputCol("sehir") \
.setOutputCol("sehir_index")

sehir_indexer_model = sehir_indexer.fit(meslek_idexer_df)
sehir_indexer_df = sehir_indexer_model.transform(meslek_idexer_df)

sehir_indexer_df.toPandas().head(5)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0,0.0
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,3.0
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0,1.0
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0,0.0
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0,2.0


In [43]:
## sehir frekansları
df1.groupBy("sehir") \
        .agg(F.count("sehir").alias("sayi")) \
        .sort(F.desc("sayi")) \
        .toPandas().head(10)

,sehir,sayi
0,Ankara,7
1,Istanbul,4
2,Çorum,1
3,Kayseri,1
4,Bursa,1
5,İzmir,1


### 2. OneHotEncoder AŞAMASI

In [44]:
encoder = OneHotEncoder() \
    .setInputCols(["meslek_index","sehir_index"]) \
    .setOutputCols(["meslek_encoded","sehir_encoded"])

encoder_model = encoder.fit(sehir_indexer_df)
encoder_df = encoder_model.transform(sehir_indexer_df)


encoder_df.toPandas().head(5)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,3.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)"
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)"


### 3. VectorAssembler AŞAMASI

In [45]:
assembler = VectorAssembler() \
    .setInputCols(["yas","aylik_gelir","meslek_encoded","sehir_encoded"]) \
    .setOutputCol("vectorized_features")

# Pandas dataframe head() metodundan satır truncate'i kaldırmak için 
import pandas as pd
pd.set_option('display.max_colwidth', -1)

vector_df = assembler.transform(encoder_df)
vector_df.select("vectorized_features").toPandas().head(15)

/Users/resitkadir/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


,vectorized_features
0,"(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
1,"(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)"
2,"(30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
3,"(29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
4,"(23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)"
5,"(33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
6,"(29.0, 7300.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
7,"(31.0, 12000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
8,"(33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
9,"(46.0, 12000.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"


### 4. LabelIndexer AŞAMASI

In [46]:
label_indexer = StringIndexer() \
.setInputCol("ekonomik_durum") \
.setOutputCol("label")

label_indexer_model = label_indexer.fit(vector_df)
label_indexer_df = label_indexer_model.transform(vector_df)

label_indexer_df.select("vectorized_features","label").toPandas().head(15)



,vectorized_features,label
0,"(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
1,"(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)",0.0
2,"(30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0
3,"(29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
4,"(23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",0.0
5,"(33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
6,"(29.0, 7300.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0
7,"(31.0, 12000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0
8,"(33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",1.0
9,"(46.0, 12000.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0


### 5. StandardScaler AŞAMASI

In [47]:
scaler = StandardScaler() \
    .setInputCol("vectorized_features") \
    .setOutputCol("features")

scaler_model = scaler.fit(label_indexer_df)
scaler_df = scaler_model.transform(label_indexer_df)

scaler_df.select("features","label").toPandas().head()

,features,label
0,"(5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)",0.0
1,"(6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0)",0.0
2,"(4.292812263480104, 1.9859783574511787, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)",1.0
3,"(4.149718521364101, 0.9267899001438834, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)",0.0
4,"(3.29115606866808, 1.0591884573072952, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0)",0.0


### 6. Train-Test Ayırma AŞAMASI

In [48]:
train_df, test_df = scaler_df.randomSplit([0.7,0.3], seed=142)
train_df.toPandas().head(5)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded,vectorized_features,label,features
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
1,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.292812263480104, 1.9859783574511787, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
2,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)","(23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",0.0,"(3.29115606866808, 1.0591884573072952, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0)"
3,6,Ali,33,Memur,Ankara,4250,kötü,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(4.722093489828115, 0.9378231132408343, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
4,8,Murat,31,Müzisyen,Istanbul,12000,iyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(31.0, 12000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.435906005596108, 2.6479711432682382, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"


In [49]:
test_df.toPandas().head(5)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded,vectorized_features,label,features
0,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,3.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)",0.0,"(6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0)"
1,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(4.149718521364101, 0.9267899001438834, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
2,7,Dilek,29,Pazarlamaci,Istanbul,7300,iyi,2.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(29.0, 7300.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.149718521364101, 1.610849112154845, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
3,9,Ahmet,33,Doktor,Ankara,18000,iyi,3.0,0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",1.0,"(4.722093489828115, 3.9719567149023574, 0.0, 0.0, 0.0, 2.8419928002940256, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
4,11,Hicaziye,47,Tuhafiyeci,Ankara,4800,kötü,7.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(47.0, 4800.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(6.725405879452164, 1.0591884573072952, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"


### 7. Basit Bir Makine Öğrenmesi Modeli

In [50]:
from pyspark.ml.classification import LogisticRegression

lr_object = LogisticRegression() \
            .setFeaturesCol("features") \
            .setLabelCol("label") \
            .setPredictionCol("prediction")

lr_model = lr_object.fit(train_df)

result_df = lr_model.transform(test_df)

result_df.select("label","prediction").toPandas().head()

,label,prediction
0,0.0,0.0
1,0.0,0.0
2,1.0,1.0
3,1.0,1.0
4,0.0,0.0


In [51]:
spark.stop()

# PipelineOps <a class="anchor" id="4"></a>

In [52]:
import findspark
findspark.init("/Users/resitkadir/spark/spark-3.0.0/")


from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline, PipelineModel


spark = SparkSession.builder \
        .appName("PipelineOps") \
        .master("local[2]") \
        .config("spark.executor.memory","4g") \
        .config("spark.driver.memory","2g") \
        .getOrCreate()


df = spark.read \
        .option("header","True") \
        .option("sep",",") \
        .option("inferSchema","True") \
        .format("csv") \
        .load("datasets/simple_data.csv")

df.toPandas().head(3)

,sirano,isim,yas,meslek,sehir,aylik_gelir
0,1,Cemal,35,Isci,Ankara,3500
1,2,Ceyda,42,Memur,Kayseri,4200
2,3,Timur,30,Müzisyen,Istanbul,9000


In [53]:
# VERİ SETİNE ETİKET EKLEME
df1 = df.withColumn("ekonomik_durum",
    F.when(F.col("aylik_gelir") > 7000, "iyi").otherwise("kötü"))

train_df, test_df = df1.randomSplit([0.8, 0.2], seed=142)

train_df.show()

+------+--------+---+-----------+--------+-----------+--------------+
|sirano|    isim|yas|     meslek|   sehir|aylik_gelir|ekonomik_durum|
+------+--------+---+-----------+--------+-----------+--------------+
|     1|   Cemal| 35|       Isci|  Ankara|       3500|          kötü|
|     2|   Ceyda| 42|      Memur| Kayseri|       4200|          kötü|
|     3|   Timur| 30|   Müzisyen|Istanbul|       9000|           iyi|
|     5| Yasemin| 23|Pazarlamaci|   Bursa|       4800|          kötü|
|     6|     Ali| 33|      Memur|  Ankara|       4250|          kötü|
|     7|   Dilek| 29|Pazarlamaci|Istanbul|       7300|           iyi|
|     8|   Murat| 31|   Müzisyen|Istanbul|      12000|           iyi|
|    10|Muhittin| 46|     Berber|Istanbul|      12000|           iyi|
|    12|   Harun| 43|    Tornacı|  Ankara|       4200|          kötü|
|    13|   Hakkı| 33|      Memur|   Çorum|       3750|          kötü|
|    14| Gülizar| 37|     Doktor|   İzmir|      14250|           iyi|
+------+--------+---

### Pipeline için nesneleri oluşturma

In [54]:
meslek_indexer = StringIndexer() \
        .setInputCol("meslek") \
        .setOutputCol("meslek_index") \
        .setHandleInvalid("skip")

sehir_indexer = StringIndexer() \
                .setInputCol("sehir") \
                .setOutputCol("sehir_index") \
                .setHandleInvalid("skip")

encoder = OneHotEncoder() \
            .setInputCols(["meslek_index","sehir_index"]) \
            .setOutputCols(["meslek_encoded","sehir_encoded"])

assembler = VectorAssembler() \
            .setInputCols(["yas","aylik_gelir","meslek_encoded","sehir_encoded"]) \
            .setOutputCol("vectorized_features")

label_indexer = StringIndexer() \
                .setInputCol("ekonomik_durum") \
                .setOutputCol("label")

scaler = StandardScaler() \
            .setInputCol("vectorized_features") \
            .setOutputCol("features")

lr_object = LogisticRegression() \
            .setFeaturesCol("features") \
            .setLabelCol("label") \
            .setPredictionCol("prediction")

In [55]:
meslek_indexer

StringIndexer_6e587032f83e

# pipeLine

In [56]:
pipeline_nesnesi = Pipeline() \
                  .setStages([meslek_indexer,sehir_indexer,encoder,assembler,label_indexer,scaler,lr_object])



In [57]:
pipeline_modeli = pipeline_nesnesi.fit(train_df)

In [58]:
pipeline_modeli.transform(test_df).select("label","prediction").toPandas().head()

,label,prediction
0,0.0,0.0
1,1.0,1.0
2,1.0,0.0


In [59]:
spark.stop()